In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import time
import random

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import SVC

NUM_OF_ROWS = 12500
ATTACK_NAME = 'DoS'

In [3]:
pd.set_option('display.max_columns', None)  # Show all columns

---

In [4]:
# import the attack sample dataset
dosSamples = pd.read_csv('dos_samples_1.csv')
print(f'Dataset Shape: {dosSamples.shape}')
dosSamples

Dataset Shape: (18, 26)


,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,1,60.0,60,60,0.0,0.0,259948,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,57.282503,9998,0,0,2.773013,3605.464376,1.589317,0.000277,0.019216
1,1,60.0,60,60,0.0,0.0,259922,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,31.935373,9997,0,0,4.922741,2030.779220,1.713713,0.000492,0.028124
2,1,60.0,60,60,0.0,0.0,259974,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,30.416988,9999,0,0,5.267316,1898.310308,1.850942,0.000527,0.030091
3,1,60.0,60,60,0.0,0.0,236886,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,26.008564,9111,0,0,44.188204,206.186248,44.076651,0.004851,0.461770
4,1,60.0,60,60,0.0,0.0,256802,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,26.036906,9877,0,0,161.398189,61.196473,161.285302,0.016342,1.622864
5,1,60.0,60,60,0.0,0.0,259922,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,60.236848,9997,0,0,2.909270,3436.257430,1.548943,0.000291,0.019841
6,1,60.0,60,60,0.0,0.0,256568,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,26.013181,9868,0,0,81.048627,121.754068,80.927725,0.008214,0.814672
7,1,60.0,60,60,0.0,0.0,259974,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,64.413776,9999,0,0,2.683720,3725.798694,1.323679,0.000268,0.018444
8,1,60.0,60,60,0.0,0.0,259610,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,26.015633,9985,0,0,101.352469,98.517580,101.220045,0.010151,1.012960
9,1,60.0,60,60,0.0,0.0,259792,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,26.013017,9992,0,0,45.536350,219.429093,45.409360,0.004558,0.454275


In [5]:
# find the columns that we need to synthesis data for to produce an attack dataset
columnsToGather = dosSamples.replace(0, np.nan) #replace all 0 values with null
columnsToGather = columnsToGather.dropna(how="all", axis=1).columns.tolist() #remove all columns where there are null values
columnsToGather #left with all columns that the values are not 0 (be know for a fact that the data is consistant and there are not missing values in the rows)

['Number of Ports',
 'Average Packet Length',
 'Packet Length Min',
 'Packet Length Max',
 'Total Length of Fwd Packet',
 'Fwd Packet Length Max',
 'Fwd Packet Length Mean',
 'Fwd Packet Length Min',
 'Fwd Segment Size Avg',
 'Subflow Fwd Bytes',
 'SYN Flag Count',
 'Flow Duration',
 'Packets Per Second',
 'IAT Max',
 'IAT Mean',
 'IAT Std']

In [6]:
# find an approximate minimum and maximum values of each column and save that data into a dictionary
MinMaxDict = {col: (dosSamples[col].min() * 0.85, dosSamples[col].max() * 1.1) for col in columnsToGather}
MinMaxDict['Number of Ports'] = (1, 1)
MinMaxDict

{'Number of Ports': (1, 1),
 'Average Packet Length': (np.float64(51.0), np.float64(66.0)),
 'Packet Length Min': (np.float64(51.0), np.float64(66.0)),
 'Packet Length Max': (np.float64(51.0), np.float64(66.0)),
 'Total Length of Fwd Packet': (np.float64(59891.0), np.float64(286000.0)),
 'Fwd Packet Length Max': (np.float64(22.099999999999998), np.float64(28.6)),
 'Fwd Packet Length Mean': (np.float64(22.099999999999998), np.float64(28.6)),
 'Fwd Packet Length Min': (np.float64(22.099999999999998), np.float64(28.6)),
 'Fwd Segment Size Avg': (np.float64(5.1), np.float64(6.6000000000000005)),
 'Subflow Fwd Bytes': (np.float64(22.107279314888007),
  np.float64(494.80968858131496)),
 'SYN Flag Count': (np.float64(2303.5), np.float64(11000.0)),
 'Flow Duration': (np.float64(1.3379797458648681),
  np.float64(177.53800797462466)),
 'Packets Per Second': (np.float64(25.737769126305732),
  np.float64(6988.14763743391)),
 'IAT Max': (np.float64(1.125127124786377), np.float64(177.41383211612703)

In [7]:
# change values to int for the columns that should be int (because when calculating approximate values they will be turned into floats)
intColumns = ['SYN Flag Count']
for key, val in MinMaxDict.items():
    if key in intColumns:
        MinMaxDict[key] = tuple([int(v) for v in val])
MinMaxDict

{'Number of Ports': (1, 1),
 'Average Packet Length': (np.float64(51.0), np.float64(66.0)),
 'Packet Length Min': (np.float64(51.0), np.float64(66.0)),
 'Packet Length Max': (np.float64(51.0), np.float64(66.0)),
 'Total Length of Fwd Packet': (np.float64(59891.0), np.float64(286000.0)),
 'Fwd Packet Length Max': (np.float64(22.099999999999998), np.float64(28.6)),
 'Fwd Packet Length Mean': (np.float64(22.099999999999998), np.float64(28.6)),
 'Fwd Packet Length Min': (np.float64(22.099999999999998), np.float64(28.6)),
 'Fwd Segment Size Avg': (np.float64(5.1), np.float64(6.6000000000000005)),
 'Subflow Fwd Bytes': (np.float64(22.107279314888007),
  np.float64(494.80968858131496)),
 'SYN Flag Count': (2303, 11000),
 'Flow Duration': (np.float64(1.3379797458648681),
  np.float64(177.53800797462466)),
 'Packets Per Second': (np.float64(25.737769126305732),
  np.float64(6988.14763743391)),
 'IAT Max': (np.float64(1.125127124786377), np.float64(177.41383211612703)),
 'IAT Mean': (np.float64(

In [8]:
# creating an empty dataframe before adding values to it
dosDataset = pd.DataFrame(np.zeros((NUM_OF_ROWS, len(dosSamples.columns))), columns=dosSamples.columns)
dosDataset.head(3)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# adding zeros to all columns that should not have any values
zeroColumns = [col for col in dosSamples.columns if col not in columnsToGather]
for col in zeroColumns:
    dosDataset[col] = int(0)
zeroColumns

['Packet Length Std',
 'Packet Length Variance',
 'Fwd Packet Length Std',
 'Bwd Packet Length Max',
 'Bwd Packet Length Mean',
 'Bwd Packet Length Min',
 'Bwd Packet Length Std',
 'Bwd Segment Size Avg',
 'ACK Flag Count',
 'RST Flag Count']

In [10]:
dosDataset.head(3)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0


### Same values

In [11]:
same_value = ['Average Packet Length', 'Packet Length Min', 'Packet Length Max']
val = np.random.randint(MinMaxDict[same_value[0]][0], MinMaxDict[same_value[0]][1]*1.1, NUM_OF_ROWS)

for col in same_value:
    dosDataset[col] = val

In [12]:
same_value2 = ['Fwd Packet Length Max', 'Fwd Packet Length Mean', 'Fwd Packet Length Min']
val2 = np.random.randint(MinMaxDict[same_value2[0]][0], MinMaxDict[same_value2[0]][1]*1.25, NUM_OF_ROWS)

for col in same_value2:
    dosDataset[col] = val2

In [13]:
dosDataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,58,58,58,0,0,0.0,30,30,30,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
1,0.0,55,55,55,0,0,0.0,27,27,27,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
2,0.0,55,55,55,0,0,0.0,27,27,27,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
3,0.0,64,64,64,0,0,0.0,32,32,32,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
4,0.0,57,57,57,0,0,0.0,33,33,33,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,0.0,63,63,63,0,0,0.0,31,31,31,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
12496,0.0,69,69,69,0,0,0.0,32,32,32,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
12497,0.0,60,60,60,0,0,0.0,34,34,34,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
12498,0.0,55,55,55,0,0,0.0,27,27,27,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0


In [14]:
dosDataset['Fwd Segment Size Avg'] = np.random.randint(MinMaxDict['Fwd Segment Size Avg'][0]*0.9, MinMaxDict['Fwd Segment Size Avg'][1]*1.5, NUM_OF_ROWS)
dosDataset['Number of Ports'] = np.full(shape=NUM_OF_ROWS, fill_value=1, dtype=int)

In [15]:
rand_values = dosDataset['SYN Flag Count'] = np.random.randint(MinMaxDict['SYN Flag Count'][0], MinMaxDict['SYN Flag Count'][1]*1.1, NUM_OF_ROWS)
usual_values = np.random.randint(8176, 10658, NUM_OF_ROWS)

# Choose values randomly (10% from rand_values, 90% from usual_values)
chosen_values = np.where(np.random.rand(NUM_OF_ROWS) > 0.1, usual_values, rand_values) 

dosDataset['SYN Flag Count'] = chosen_values

In [16]:
rand_values = np.random.uniform(MinMaxDict['Flow Duration'][0], MinMaxDict['Flow Duration'][1], NUM_OF_ROWS)
usual_values = np.random.uniform(1.654, 45.175, NUM_OF_ROWS)

# Choose values randomly (25% from rand_values, 75% from usual_values)
chosen_values = np.where(np.random.rand(NUM_OF_ROWS) > 0.25, usual_values, rand_values) 

dosDataset['Flow Duration'] = chosen_values

In [17]:
rand_values = np.random.uniform(MinMaxDict['Subflow Fwd Bytes'][0], MinMaxDict['Subflow Fwd Bytes'][1], NUM_OF_ROWS)
usual_values = np.random.uniform(13.763, 72.146, NUM_OF_ROWS)

# Choose values randomly (10% from rand_values, 90% from usual_values)
chosen_values = np.where(np.random.rand(NUM_OF_ROWS) > 0.1, usual_values, rand_values) 

dosDataset['Subflow Fwd Bytes'] = chosen_values

In [18]:
dosDataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,1,58,58,58,0,0,0.0,30,30,30,0,0,0,0,0,7,0,431.214619,10225,0,0,31.043445,0.0,0.0,0.0,0.0
1,1,55,55,55,0,0,0.0,27,27,27,0,0,0,0,0,8,0,14.871573,9838,0,0,9.950436,0.0,0.0,0.0,0.0
2,1,55,55,55,0,0,0.0,27,27,27,0,0,0,0,0,8,0,29.070678,8814,0,0,27.968782,0.0,0.0,0.0,0.0
3,1,64,64,64,0,0,0.0,32,32,32,0,0,0,0,0,4,0,41.540814,8289,0,0,148.824786,0.0,0.0,0.0,0.0
4,1,57,57,57,0,0,0.0,33,33,33,0,0,0,0,0,8,0,291.659139,10714,0,0,20.128432,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,1,63,63,63,0,0,0.0,31,31,31,0,0,0,0,0,4,0,244.455211,8552,0,0,15.716316,0.0,0.0,0.0,0.0
12496,1,69,69,69,0,0,0.0,32,32,32,0,0,0,0,0,5,0,34.757067,10214,0,0,26.292172,0.0,0.0,0.0,0.0
12497,1,60,60,60,0,0,0.0,34,34,34,0,0,0,0,0,7,0,43.553852,8940,0,0,11.726504,0.0,0.0,0.0,0.0
12498,1,55,55,55,0,0,0.0,27,27,27,0,0,0,0,0,4,0,25.934305,10370,0,0,54.339571,0.0,0.0,0.0,0.0


### First Correlation

In [19]:
# finding the correlation between the Number of Ports column to the rest of the columns in order to create new data
first_correlation = ['SYN Flag Count', 'Total Length of Fwd Packet']
independent_col = dosSamples[first_correlation[0]].values.reshape(-1, 1) #column 'Number of Ports'
dependent_cols = dosSamples[first_correlation[1]].values #the rest of the columns that are not zeros

# find the scaling factors using least squares function
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond=None)[0]

scaling_factors = [(name,factor) for name, factor in zip(first_correlation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('Total Length of Fwd Packet', np.float64(26.000000000000007))


In [20]:
# adding the rest of the attack feature values to the dataset at random based on the smaple data
for index, row in dosDataset.iterrows():
    for col, factor in scaling_factors: #iterating over all rows we need to add values to except 'Number of Ports'
        delta = random.uniform(factor * 0.05, factor * 0.25) # select a dantom
        updatedFactor = factor + random.choice([-1, 1]) * delta
        dosDataset.loc[index, col] = int(row['SYN Flag Count'] * updatedFactor)

In [21]:
dosDataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,1,58,58,58,0,0,285762.0,30,30,30,0,0,0,0,0,7,0,431.214619,10225,0,0,31.043445,0.0,0.0,0.0,0.0
1,1,55,55,55,0,0,219859.0,27,27,27,0,0,0,0,0,8,0,14.871573,9838,0,0,9.950436,0.0,0.0,0.0,0.0
2,1,55,55,55,0,0,277915.0,27,27,27,0,0,0,0,0,8,0,29.070678,8814,0,0,27.968782,0.0,0.0,0.0,0.0
3,1,64,64,64,0,0,246333.0,32,32,32,0,0,0,0,0,4,0,41.540814,8289,0,0,148.824786,0.0,0.0,0.0,0.0
4,1,57,57,57,0,0,255541.0,33,33,33,0,0,0,0,0,8,0,291.659139,10714,0,0,20.128432,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,1,63,63,63,0,0,270493.0,31,31,31,0,0,0,0,0,4,0,244.455211,8552,0,0,15.716316,0.0,0.0,0.0,0.0
12496,1,69,69,69,0,0,214989.0,32,32,32,0,0,0,0,0,5,0,34.757067,10214,0,0,26.292172,0.0,0.0,0.0,0.0
12497,1,60,60,60,0,0,176042.0,34,34,34,0,0,0,0,0,7,0,43.553852,8940,0,0,11.726504,0.0,0.0,0.0,0.0
12498,1,55,55,55,0,0,301084.0,27,27,27,0,0,0,0,0,4,0,25.934305,10370,0,0,54.339571,0.0,0.0,0.0,0.0


### Second correlation

In [22]:
# finding the correlation between the Number of Ports column to the rest of the columns in order to create new data
secondCorrelation = ['Flow Duration', 'Packets Per Second', 'IAT Max', 'IAT Mean', 'IAT Std']
independent_col = dosSamples[secondCorrelation[0]].values.reshape(-1, 1) #column 'Number of Ports'
dependent_cols = dosSamples[secondCorrelation[1:]].values #the rest of the columns that are not zeros

# find the scaling factors using least squares function
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond=None)[0]

scaling_factors = [(name,factor) for name, factor in zip(secondCorrelation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

durationToPacketsCorr = [x * y for x, y in zip(dosSamples['Flow Duration'].values, dosSamples['Packets Per Second'].values)]
durationToPacketsCorr = np.mean(durationToPacketsCorr)
durationToPacketsCorr

('Packets Per Second', np.float64(1.8396896437579322))
('IAT Max', np.float64(0.998429144907052))
('IAT Mean', np.float64(0.00015181064048409737))
('IAT Std', np.float64(0.01191782011351499))


np.float64(8585.611111111111)

In [23]:
# adding the rest of the attack feature values to the dataset at random based on the smaple data
for index, row in dosDataset.iterrows():
    for col, factor in scaling_factors: #iterating over all rows we need to add values to except 'Number of Ports'
        # calculate a random small delta of the factor for adding some randomness
        if col == 'Packets Per Second':
            delta = random.uniform(durationToPacketsCorr * 0.1, durationToPacketsCorr * 0.15) # select a delta
            updatedFactor = durationToPacketsCorr + random.choice([-1, 1]) * delta
            dosDataset.loc[index, col] = updatedFactor / row['Flow Duration']
        else:
            if col == 'IAT Std':
                delta = random.uniform(factor * 0.1, factor * 0.35)
                updatedFactor = factor + random.choice([-1, 1]) * delta  
            elif col == 'IAT Max':
                delta = random.uniform(factor * 0.1, factor * 0.225)
                updatedFactor = factor + random.choice([-1, 1]) * delta  
            else:
                delta = random.uniform(factor * 0.05, factor * 0.2) # select a delta
                updatedFactor = factor + random.choice([-1, 1]) * delta
            dosDataset.loc[index, col] = row['Flow Duration'] * updatedFactor

In [24]:
dosDataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,1,58,58,58,0,0,285762.0,30,30,30,0,0,0,0,0,7,0,431.214619,10225,0,0,31.043445,315.499145,36.286037,0.003959,0.317898
1,1,55,55,55,0,0,219859.0,27,27,27,0,0,0,0,0,8,0,14.871573,9838,0,0,9.950436,975.593144,8.774895,0.001240,0.131743
2,1,55,55,55,0,0,277915.0,27,27,27,0,0,0,0,0,8,0,29.070678,8814,0,0,27.968782,339.231906,23.350982,0.003532,0.432179
3,1,64,64,64,0,0,246333.0,32,32,32,0,0,0,0,0,4,0,41.540814,8289,0,0,148.824786,51.574264,174.234391,0.019590,2.067112
4,1,57,57,57,0,0,255541.0,33,33,33,0,0,0,0,0,8,0,291.659139,10714,0,0,20.128432,381.561099,16.604115,0.003429,0.316821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,1,63,63,63,0,0,270493.0,31,31,31,0,0,0,0,0,4,0,244.455211,8552,0,0,15.716316,604.627257,17.985540,0.002025,0.134079
12496,1,69,69,69,0,0,214989.0,32,32,32,0,0,0,0,0,5,0,34.757067,10214,0,0,26.292172,374.401372,30.199075,0.003755,0.264038
12497,1,60,60,60,0,0,176042.0,34,34,34,0,0,0,0,0,7,0,43.553852,8940,0,0,11.726504,806.788022,14.035318,0.001518,0.099290
12498,1,55,55,55,0,0,301084.0,27,27,27,0,0,0,0,0,4,0,25.934305,10370,0,0,54.339571,137.637917,48.479560,0.009749,0.718982


In [25]:
# adding a label to the dataset
dosDataset['Label'] = ATTACK_NAME

In [26]:
dosSamples.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,18.0,18.0,18.0,18.0,18.0,18.0,18.000000,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.000000,18.000000,18.0,18.0,18.000000,18.000000,18.000000,18.000000,18.000000
mean,1.0,60.0,60.0,60.0,0.0,0.0,223225.888889,26.0,26.0,26.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,58.869588,8585.611111,0.0,0.0,50.730569,1261.620636,50.075167,0.007958,0.609617
std,0.0,0.0,0.0,0.0,0.0,0.0,67405.300285,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,98.729675,2592.511549,0.0,0.0,47.078915,1854.329691,47.670273,0.009044,0.595760
min,1.0,60.0,60.0,60.0,0.0,0.0,70460.000000,26.0,26.0,26.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,26.008564,2710.000000,0.0,0.0,1.574094,30.279728,1.323679,0.000157,0.014803
25%,1.0,60.0,60.0,60.0,0.0,0.0,240935.500000,26.0,26.0,26.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,26.015761,9266.750000,0.0,0.0,5.008885,97.571008,1.748020,0.000501,0.028616
50%,1.0,60.0,60.0,60.0,0.0,0.0,257478.000000,26.0,26.0,26.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,26.061350,9903.000000,0.0,0.0,44.862277,212.807671,44.743005,0.004704,0.458022
75%,1.0,60.0,60.0,60.0,0.0,0.0,259922.000000,26.0,26.0,26.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,52.472023,9997.000000,0.0,0.0,75.907850,1997.661992,75.807678,0.010250,1.020746
max,1.0,60.0,60.0,60.0,0.0,0.0,260000.000000,26.0,26.0,26.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,449.826990,10000.000000,0.0,0.0,161.398189,6352.861489,161.285302,0.033034,1.999864


In [27]:
dosDataset.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,12500.0,12500.00000,12500.00000,12500.00000,12500.0,12500.0,12500.000000,12500.000000,12500.000000,12500.000000,12500.0,12500.0,12500.0,12500.0,12500.0,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.0,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000
mean,1.0,61.06600,61.06600,61.06600,0.0,0.0,238127.033520,27.967520,27.967520,27.967520,0.0,0.0,0.0,0.0,0.0,6.013120,0.0,64.589734,9181.052960,0.0,0.0,40.729919,543.321943,40.551447,0.006180,0.485324
std,0.0,6.08815,6.08815,6.08815,0.0,0.0,52052.032481,3.750551,3.750551,3.750551,0.0,0.0,0.0,0.0,0.0,1.417769,0.0,79.864332,1321.865511,0.0,0.0,40.331478,726.103599,41.249774,0.006203,0.508961
min,1.0,51.00000,51.00000,51.00000,0.0,0.0,48583.000000,22.000000,22.000000,22.000000,0.0,0.0,0.0,0.0,0.0,4.000000,0.0,13.765768,2318.000000,0.0,0.0,1.342013,41.394498,1.234898,0.000184,0.013821
25%,1.0,56.00000,56.00000,56.00000,0.0,0.0,202298.500000,25.000000,25.000000,25.000000,0.0,0.0,0.0,0.0,0.0,5.000000,0.0,29.743480,8671.000000,0.0,0.0,15.295727,192.599017,14.901704,0.002266,0.171685
50%,1.0,61.00000,61.00000,61.00000,0.0,0.0,237588.500000,28.000000,28.000000,28.000000,0.0,0.0,0.0,0.0,0.0,6.000000,0.0,45.949268,9352.000000,0.0,0.0,28.591204,299.091156,27.898276,0.004277,0.321449
75%,1.0,66.00000,66.00000,66.00000,0.0,0.0,278854.750000,31.000000,31.000000,31.000000,0.0,0.0,0.0,0.0,0.0,7.000000,0.0,61.795565,10025.000000,0.0,0.0,42.366196,561.906704,45.086426,0.006700,0.554217
max,1.0,71.00000,71.00000,71.00000,0.0,0.0,387708.000000,34.000000,34.000000,34.000000,0.0,0.0,0.0,0.0,0.0,8.000000,0.0,493.906710,12094.000000,0.0,0.0,177.519719,7100.433680,215.515610,0.031813,2.785792


In [28]:
dosSamples

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,1,60.0,60,60,0.0,0.0,259948,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,57.282503,9998,0,0,2.773013,3605.464376,1.589317,0.000277,0.019216
1,1,60.0,60,60,0.0,0.0,259922,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,31.935373,9997,0,0,4.922741,2030.779220,1.713713,0.000492,0.028124
2,1,60.0,60,60,0.0,0.0,259974,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,30.416988,9999,0,0,5.267316,1898.310308,1.850942,0.000527,0.030091
3,1,60.0,60,60,0.0,0.0,236886,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,26.008564,9111,0,0,44.188204,206.186248,44.076651,0.004851,0.461770
4,1,60.0,60,60,0.0,0.0,256802,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,26.036906,9877,0,0,161.398189,61.196473,161.285302,0.016342,1.622864
5,1,60.0,60,60,0.0,0.0,259922,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,60.236848,9997,0,0,2.909270,3436.257430,1.548943,0.000291,0.019841
6,1,60.0,60,60,0.0,0.0,256568,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,26.013181,9868,0,0,81.048627,121.754068,80.927725,0.008214,0.814672
7,1,60.0,60,60,0.0,0.0,259974,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,64.413776,9999,0,0,2.683720,3725.798694,1.323679,0.000268,0.018444
8,1,60.0,60,60,0.0,0.0,259610,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,26.015633,9985,0,0,101.352469,98.517580,101.220045,0.010151,1.012960
9,1,60.0,60,60,0.0,0.0,259792,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,26.013017,9992,0,0,45.536350,219.429093,45.409360,0.004558,0.454275


In [29]:
x = dosDataset[dosDataset['Flow Duration'] > 30]
x[x['Flow Duration'] < 31][:20]

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std,Label
39,1,56,56,56,0,0,164619.0,27,27,27,0,0,0,0,0,7,0,32.899644,8307,0,0,30.927247,237.926832,26.273679,0.004438,0.291452,DoS
61,1,58,58,58,0,0,206888.0,34,34,34,0,0,0,0,0,7,0,18.413787,9156,0,0,30.832136,248.079359,36.157734,0.004417,0.310354,DoS
141,1,56,56,56,0,0,185518.0,32,32,32,0,0,0,0,0,5,0,25.604206,6401,0,0,30.802781,315.381998,27.063979,0.004036,0.452611,DoS
284,1,54,54,54,0,0,240129.0,33,33,33,0,0,0,0,0,6,0,20.290843,8744,0,0,30.450785,311.996590,33.977535,0.004057,0.488983,DoS
482,1,68,68,68,0,0,233173.0,27,27,27,0,0,0,0,0,5,0,56.670178,7299,0,0,30.542776,247.003070,36.219129,0.004901,0.427685,DoS
505,1,67,67,67,0,0,261616.0,34,34,34,0,0,0,0,0,8,0,19.674976,8323,0,0,30.340023,247.553391,33.993256,0.005187,0.466788,DoS
609,1,53,53,53,0,0,248164.0,31,31,31,0,0,0,0,0,8,0,48.913056,10407,0,0,30.223591,315.051058,23.487788,0.005139,0.409253,DoS
634,1,66,66,66,0,0,176169.0,23,23,23,0,0,0,0,0,6,0,22.200304,8177,0,0,30.649162,239.367664,34.190402,0.005279,0.288751,DoS
658,1,62,62,62,0,0,246195.0,34,34,34,0,0,0,0,0,6,0,57.887653,8818,0,0,30.208672,243.929285,25.855643,0.004828,0.436078,DoS
674,1,53,53,53,0,0,248565.0,27,27,27,0,0,0,0,0,5,0,42.594168,10426,0,0,30.741166,239.886456,37.086842,0.004315,0.427045,DoS


In [30]:
x = dosDataset[dosDataset['Flow Duration'] > 2.6]
x[x['Flow Duration'] < 2.7]

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std,Label
295,1,61,61,61,0,0,274882.0,26,26,26,0,0,0,0,0,4,0,50.622530,9064,0,0,2.641013,3663.081127,3.118698,0.000471,0.025873,DoS
1100,1,67,67,67,0,0,209150.0,34,34,34,0,0,0,0,0,8,0,34.972889,10183,0,0,2.690038,3615.101687,2.113142,0.000347,0.037740,DoS
1573,1,71,71,71,0,0,286200.0,22,22,22,0,0,0,0,0,7,0,50.596617,10025,0,0,2.602003,3647.963783,3.175394,0.000459,0.040499,DoS
1814,1,60,60,60,0,0,204715.0,27,27,27,0,0,0,0,0,6,0,45.809994,9162,0,0,2.679954,3636.401426,2.235339,0.000383,0.023171,DoS
3612,1,65,65,65,0,0,204342.0,34,34,34,0,0,0,0,0,4,0,44.547996,9162,0,0,2.608861,3712.485195,2.943691,0.000473,0.022290,DoS
4434,1,62,62,62,0,0,267978.0,33,33,33,0,0,0,0,0,8,0,64.075289,8840,0,0,2.626158,2844.419565,3.071329,0.000369,0.039410,DoS
5844,1,60,60,60,0,0,285217.0,32,32,32,0,0,0,0,0,4,0,24.228089,8929,0,0,2.619113,3651.564734,2.892248,0.000374,0.034855,DoS
6636,1,61,61,61,0,0,330431.0,32,32,32,0,0,0,0,0,4,0,63.097112,10434,0,0,2.661307,3576.049209,2.259151,0.000374,0.042688,DoS
7701,1,55,55,55,0,0,237851.0,31,31,31,0,0,0,0,0,8,0,58.697064,9960,0,0,2.624509,2811.918733,3.204028,0.000361,0.025539,DoS
8028,1,54,54,54,0,0,215652.0,25,25,25,0,0,0,0,0,7,0,47.297022,9385,0,0,2.612082,3733.491459,2.919577,0.000470,0.038070,DoS


---

### Adding rows with low flow duration

In [31]:
NUM_OF_ROWS = 6250

In [32]:
# import the attack sample dataset
dosSamples = pd.read_csv('dos_samples_2.csv')
print(f'Dataset Shape: {dosSamples.shape}')
dosSamples

Dataset Shape: (8, 26)


,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,1,60.0,60,60,0.0,0.0,251472,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9672,0,0,0.113266,85391.915937,0.002451,0.000012,0.000077
1,1,60.0,60,60,0.0,0.0,259818,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9993,0,0,0.125600,79562.038842,0.013888,0.000013,0.000176
2,1,60.0,60,60,0.0,0.0,259558,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9983,0,0,0.122131,81740.025636,0.001796,0.000012,0.000070
3,1,60.0,60,60,0.0,0.0,250952,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9652,0,0,0.155204,62189.191252,0.002296,0.000016,0.000083
4,1,60.0,60,60,0.0,0.0,259844,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9994,0,0,0.150346,66473.316835,0.005427,0.000015,0.000100
5,1,60.0,60,60,0.0,0.0,258440,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9940,0,0,0.146744,67737.005470,0.001699,0.000015,0.000080
6,1,60.0,60,60,0.0,0.0,253734,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9759,0,0,0.133626,73032.202973,0.003685,0.000014,0.000085
7,1,60.0,60,60,0.0,0.0,259740,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9990,0,0,0.150142,66537.031827,0.013583,0.000015,0.000234


In [33]:
# find an approximate minimum and maximum values of each column and save that data into a dictionary
MinMaxDict = {col: (dosSamples[col].min() * 0.85, dosSamples[col].max() * 1.1) for col in columnsToGather}
MinMaxDict['Number of Ports'] = (1, 1)
MinMaxDict

{'Number of Ports': (1, 1),
 'Average Packet Length': (np.float64(51.0), np.float64(66.0)),
 'Packet Length Min': (np.float64(51.0), np.float64(66.0)),
 'Packet Length Max': (np.float64(51.0), np.float64(66.0)),
 'Total Length of Fwd Packet': (np.float64(213309.19999999998),
  np.float64(285828.4)),
 'Fwd Packet Length Max': (np.float64(22.099999999999998), np.float64(28.6)),
 'Fwd Packet Length Mean': (np.float64(22.099999999999998), np.float64(28.6)),
 'Fwd Packet Length Min': (np.float64(22.099999999999998), np.float64(28.6)),
 'Fwd Segment Size Avg': (np.float64(5.1), np.float64(6.6000000000000005)),
 'Subflow Fwd Bytes': (np.float64(0.0), np.float64(0.0)),
 'SYN Flag Count': (np.float64(8204.199999999999),
  np.float64(10993.400000000001)),
 'Flow Duration': (np.float64(0.09627609252929688),
  np.float64(0.17072420120239254)),
 'Packets Per Second': (np.float64(52860.81256459571),
  np.float64(93931.107530648)),
 'IAT Max': (np.float64(0.00144432783126828),
  np.float64(0.01527693

In [34]:
# change values to int for the columns that should be int (because when calculating approximate values they will be turned into floats)
intColumns = ['SYN Flag Count']
for key, val in MinMaxDict.items():
    if key in intColumns:
        MinMaxDict[key] = tuple([int(v) for v in val])
MinMaxDict

{'Number of Ports': (1, 1),
 'Average Packet Length': (np.float64(51.0), np.float64(66.0)),
 'Packet Length Min': (np.float64(51.0), np.float64(66.0)),
 'Packet Length Max': (np.float64(51.0), np.float64(66.0)),
 'Total Length of Fwd Packet': (np.float64(213309.19999999998),
  np.float64(285828.4)),
 'Fwd Packet Length Max': (np.float64(22.099999999999998), np.float64(28.6)),
 'Fwd Packet Length Mean': (np.float64(22.099999999999998), np.float64(28.6)),
 'Fwd Packet Length Min': (np.float64(22.099999999999998), np.float64(28.6)),
 'Fwd Segment Size Avg': (np.float64(5.1), np.float64(6.6000000000000005)),
 'Subflow Fwd Bytes': (np.float64(0.0), np.float64(0.0)),
 'SYN Flag Count': (8204, 10993),
 'Flow Duration': (np.float64(0.09627609252929688),
  np.float64(0.17072420120239254)),
 'Packets Per Second': (np.float64(52860.81256459571),
  np.float64(93931.107530648)),
 'IAT Max': (np.float64(0.00144432783126828),
  np.float64(0.015276932716369611)),
 'IAT Mean': (np.float64(9.95513313300

In [35]:
# creating an empty dataframe before adding values to it
dosDataset2 = pd.DataFrame(np.zeros((NUM_OF_ROWS, len(dosSamples.columns))), columns=dosSamples.columns)
dosDataset2.head(3)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
# adding zeros to all columns that should not have any values
zeroColumns = [col for col in dosSamples.columns if col not in columnsToGather]
for col in zeroColumns:
    dosDataset2[col] = int(0)
zeroColumns

['Packet Length Std',
 'Packet Length Variance',
 'Fwd Packet Length Std',
 'Bwd Packet Length Max',
 'Bwd Packet Length Mean',
 'Bwd Packet Length Min',
 'Bwd Packet Length Std',
 'Bwd Segment Size Avg',
 'ACK Flag Count',
 'RST Flag Count']

In [37]:
same_value = ['Average Packet Length', 'Packet Length Min', 'Packet Length Max']
val = np.random.randint(MinMaxDict[same_value[0]][0], MinMaxDict[same_value[0]][1]*1.1, NUM_OF_ROWS)

for col in same_value:
    dosDataset2[col] = val

In [38]:
same_value2 = ['Fwd Packet Length Max', 'Fwd Packet Length Mean', 'Fwd Packet Length Min']
val2 = np.random.randint(MinMaxDict[same_value2[0]][0], MinMaxDict[same_value2[0]][1]*1.25, NUM_OF_ROWS)

for col in same_value2:
    dosDataset2[col] = val2

In [39]:
dosDataset2['Fwd Segment Size Avg'] = np.random.randint(MinMaxDict['Fwd Segment Size Avg'][0]*0.9, MinMaxDict['Fwd Segment Size Avg'][1]*1.5, NUM_OF_ROWS)
dosDataset2['Flow Duration'] = np.random.uniform(MinMaxDict['Flow Duration'][0]*0.95, MinMaxDict['Flow Duration'][1]*1.05, NUM_OF_ROWS)
dosDataset2['Number of Ports'] = np.full(shape=NUM_OF_ROWS, fill_value=1, dtype=int)
dosDataset2['Subflow Fwd Bytes'] = np.full(shape=NUM_OF_ROWS, fill_value=0, dtype=int)
dosDataset2['SYN Flag Count'] = np.random.randint(MinMaxDict['SYN Flag Count'][0]*0.9, MinMaxDict['SYN Flag Count'][1]*1.1, NUM_OF_ROWS)

In [40]:
# finding the correlation between the Number of Ports column to the rest of the columns in order to create new data
first_correlation = ['SYN Flag Count', 'Total Length of Fwd Packet']
independent_col = dosSamples[first_correlation[0]].values.reshape(-1, 1) #column 'Number of Ports'
dependent_cols = dosSamples[first_correlation[1]].values #the rest of the columns that are not zeros

# find the scaling factors using least squares function
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond=None)[0]

scaling_factors = [(name,factor) for name, factor in zip(first_correlation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)
    
# adding the rest of the attack feature values to the dataset at random based on the smaple data
for index, row in dosDataset2.iterrows():
    for col, factor in scaling_factors: #iterating over all rows we need to add values to except 'Number of Ports'
        delta = random.uniform(factor * 0.05, factor * 0.25) # select a dantom
        updatedFactor = factor + random.choice([-1, 1]) * delta
        dosDataset2.loc[index, col] = int(row['SYN Flag Count'] * updatedFactor)

('Total Length of Fwd Packet', np.float64(26.000000000000007))


In [41]:
# finding the correlation between the Number of Ports column to the rest of the columns in order to create new data
secondCorrelation = ['Flow Duration', 'Packets Per Second', 'IAT Max', 'IAT Mean', 'IAT Std']
independent_col = dosSamples[secondCorrelation[0]].values.reshape(-1, 1) #column 'Number of Ports'
dependent_cols = dosSamples[secondCorrelation[1:]].values #the rest of the columns that are not zeros

# find the scaling factors using least squares function
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond=None)[0]

scaling_factors = [(name,factor) for name, factor in zip(secondCorrelation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

durationToPacketsCorr = [x * y for x, y in zip(dosSamples['Flow Duration'].values, dosSamples['Packets Per Second'].values)]
durationToPacketsCorr = np.mean(durationToPacketsCorr)
durationToPacketsCorr

('Packets Per Second', np.float64(519129.6051044216))
('IAT Max', np.float64(0.040715922209246726))
('IAT Mean', np.float64(0.00010129001644309197))
('IAT Std', np.float64(0.000825544773212792))


np.float64(9872.875)

In [42]:
# adding the rest of the attack feature values to the dataset at random based on the smaple data
for index, row in dosDataset2.iterrows():
    for col, factor in scaling_factors: #iterating over all rows we need to add values to except 'Number of Ports'
        # calculate a random small delta of the factor for adding some randomness
        if col == 'Packets Per Second':
            delta = random.uniform(durationToPacketsCorr*0.025, durationToPacketsCorr * 0.075) # select a delta
            updatedFactor = durationToPacketsCorr + random.choice([-1, 1]) * delta
            dosDataset2.loc[index, col] = updatedFactor / row['Flow Duration']
        else:
            if col == 'IAT Std':
                delta = random.uniform(factor * 0.1, factor * 0.35)
                updatedFactor = factor + random.choice([-1, 1]) * delta  
            elif col == 'IAT Max':
                delta = random.uniform(factor * 0.15, factor * 0.7)
                updatedFactor = factor + random.choice([-1, 1]) * delta  
            else:
                delta = random.uniform(factor * 0.05, factor * 0.2) # select a delta
                updatedFactor = factor + random.choice([-1, 1]) * delta
            dosDataset2.loc[index, col] = row['Flow Duration'] * updatedFactor

In [43]:
dosSamples

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,1,60.0,60,60,0.0,0.0,251472,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9672,0,0,0.113266,85391.915937,0.002451,0.000012,0.000077
1,1,60.0,60,60,0.0,0.0,259818,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9993,0,0,0.125600,79562.038842,0.013888,0.000013,0.000176
2,1,60.0,60,60,0.0,0.0,259558,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9983,0,0,0.122131,81740.025636,0.001796,0.000012,0.000070
3,1,60.0,60,60,0.0,0.0,250952,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9652,0,0,0.155204,62189.191252,0.002296,0.000016,0.000083
4,1,60.0,60,60,0.0,0.0,259844,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9994,0,0,0.150346,66473.316835,0.005427,0.000015,0.000100
5,1,60.0,60,60,0.0,0.0,258440,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9940,0,0,0.146744,67737.005470,0.001699,0.000015,0.000080
6,1,60.0,60,60,0.0,0.0,253734,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9759,0,0,0.133626,73032.202973,0.003685,0.000014,0.000085
7,1,60.0,60,60,0.0,0.0,259740,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9990,0,0,0.150142,66537.031827,0.013583,0.000015,0.000234


In [44]:
dosDataset2.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,6250.0,6250.000000,6250.000000,6250.000000,6250.0,6250.0,6250.000000,6250.000000,6250.000000,6250.000000,6250.0,6250.0,6250.0,6250.0,6250.0,6250.000000,6250.0,6250.0,6250.000000,6250.0,6250.0,6250.000000,6250.000000,6250.000000,6250.000000,6250.000000
mean,1.0,61.049920,61.049920,61.049920,0.0,0.0,252804.353760,28.029600,28.029600,28.029600,0.0,0.0,0.0,0.0,0.0,6.010560,0.0,0.0,9740.070400,0.0,0.0,0.134635,76074.188777,0.005496,0.000014,0.000111
std,0.0,6.023138,6.023138,6.023138,0.0,0.0,54285.750992,3.749551,3.749551,3.749551,0.0,0.0,0.0,0.0,0.0,1.413721,0.0,0.0,1340.305056,0.0,0.0,0.025514,15449.014629,0.002746,0.000003,0.000033
min,1.0,51.000000,51.000000,51.000000,0.0,0.0,144884.000000,22.000000,22.000000,22.000000,0.0,0.0,0.0,0.0,0.0,4.000000,0.0,0.0,7383.000000,0.0,0.0,0.091474,51116.025925,0.001135,0.000007,0.000050
25%,1.0,56.000000,56.000000,56.000000,0.0,0.0,212121.250000,25.000000,25.000000,25.000000,0.0,0.0,0.0,0.0,0.0,5.000000,0.0,0.0,8603.500000,0.0,0.0,0.112147,63125.700199,0.003053,0.000011,0.000085
50%,1.0,61.000000,61.000000,61.000000,0.0,0.0,247058.000000,28.000000,28.000000,28.000000,0.0,0.0,0.0,0.0,0.0,6.000000,0.0,0.0,9735.500000,0.0,0.0,0.134036,73467.542165,0.005189,0.000013,0.000105
75%,1.0,66.000000,66.000000,66.000000,0.0,0.0,290046.500000,31.000000,31.000000,31.000000,0.0,0.0,0.0,0.0,0.0,7.000000,0.0,0.0,10881.750000,0.0,0.0,0.157000,87848.337521,0.007740,0.000016,0.000134
max,1.0,71.000000,71.000000,71.000000,0.0,0.0,390204.000000,34.000000,34.000000,34.000000,0.0,0.0,0.0,0.0,0.0,8.000000,0.0,0.0,12091.000000,0.0,0.0,0.179231,115875.527931,0.012212,0.000022,0.000198


In [45]:
x = dosDataset2[dosDataset2['Flow Duration'] > 0.16]
x[x['Flow Duration'] < 0.17][:20]

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,1,63,63,63,0,0,217942.0,32,32,32,0,0,0,0,0,5,0,0,10208,0,0,0.169591,54357.240171,0.010780,0.000020,0.000110
10,1,66,66,66,0,0,268964.0,30,30,30,0,0,0,0,0,5,0,0,11285,0,0,0.167329,62089.170234,0.003012,0.000015,0.000170
32,1,68,68,68,0,0,252605.0,31,31,31,0,0,0,0,0,8,0,0,8400,0,0,0.163866,63345.490117,0.002265,0.000015,0.000116
35,1,66,66,66,0,0,233350.0,27,27,27,0,0,0,0,0,6,0,0,10734,0,0,0.162441,56688.078987,0.005172,0.000019,0.000115
51,1,51,51,51,0,0,285175.0,32,32,32,0,0,0,0,0,4,0,0,11708,0,0,0.168188,61810.437759,0.010175,0.000015,0.000096
54,1,53,53,53,0,0,277112.0,31,31,31,0,0,0,0,0,6,0,0,11339,0,0,0.162141,63798.089252,0.004179,0.000019,0.000178
56,1,64,64,64,0,0,243403.0,23,23,23,0,0,0,0,0,6,0,0,8045,0,0,0.168510,56390.554402,0.008435,0.000019,0.000165
57,1,65,65,65,0,0,358494.0,34,34,34,0,0,0,0,0,4,0,0,11954,0,0,0.166521,55415.274977,0.009388,0.000014,0.000181
58,1,65,65,65,0,0,301000.0,22,22,22,0,0,0,0,0,6,0,0,9266,0,0,0.169041,56607.945515,0.010007,0.000015,0.000184
70,1,58,58,58,0,0,251682.0,32,32,32,0,0,0,0,0,4,0,0,11940,0,0,0.169609,60305.527119,0.008040,0.000021,0.000160


In [46]:
# sample dos attack dataset
mergedDosDataset = pd.concat([dosDataset, dosDataset2], axis=0)
mergedDosDataset = mergedDosDataset.sample(frac=1, random_state=42).reset_index(drop=True)
print(mergedDosDataset.shape)

(18750, 27)


In [ ]:
# save the dataset
# dosDataset.to_csv('dos_hping_dataset_new.csv', index=False)

---